In [ ]:
!pip install pytorch_lightning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 4.1 MB/s 
     |████████████████████████████████| 125 kB 78.4 MB/s 
     |████████████████████████████████| 512 kB 90.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 3.9 MB/s 
     |████████████████████████████████| 182 kB 63.0 MB/s 
     |████████████████████████████████| 7.6 MB 52.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/info_retrieval/')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from google.colab import drive
#from MarcoDataset import *
#from TransformerMarco import *
#from run_longformer_marco import *

import MarcoDataset 
import TransformerMarco 
import run_longformer_marco 

INFO:lightning_lite.utilities.seed:Global seed set to 42


#Getting data


In [ ]:
def prepare():
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs-lookup.tsv.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-qrels.tsv.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-queries.tsv.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-top100.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-qrels.tsv.gz

    !gzip -d msmarco-docs.tsv.gz
    !gzip -d msmarco-docs-lookup.tsv.gz
    !gzip -d msmarco-doctrain-queries.tsv.gz
    !gzip -d msmarco-doctrain-top100.gz
    !gzip -d msmarco-doctrain-qrels.tsv.gz
    !gzip -d msmarco-docdev-queries.tsv.gz
    !gzip -d msmarco-docdev-top100.gz
    !gzip -d msmarco-docdev-qrels.tsv.gz

    !cp -r "/content/info-retrieval" "/content/drive/MyDrive/info-retrieval"
    !cp -r "/content/MarcoDataset.py" "/content/drive/MyDrive/info-retrieval"
    !cp -r "/content/TransformerMarco.py" "/content/drive/MyDrive/info-retrieval"
    !cp -r "/content/run_longformer_marco.py" "/content/drive/MyDrive/info-retrieval"

#Training

In [ ]:
from argparse import Namespace
import copy
hparams_base = {
    'max_seq_len':4096,
    'lr':3e-5,
    'num_warmup_steps':1,
    'num_training_steps':120000,
    'val_check_interval':250,
    'clf_dropout':-1.0,
    'num_workers':0,
    'data_dir':'drive/MyDrive/info_retrieval/data/',
    'trainer_batch_size':5,
    'data_loader_bs':1,
    'val_data_loader_bs':1,
    'use_10_percent_of_dev':1,
    'use_wandb':0,
    'use_tensorboard':0,
    'gpus':1,
    'num_nodes':1,
    'model_name':'allenai/longformer-base-4096',
    'epochs':6,
    'run_name':'second_round/80_qrs',
    'n_tr_qrs':80,
    'res_dir':'drive/MyDrive/info_retrieval/results/'
}
for n_q in [80]:
    print(f"Training on {n_q} queries")
    hparams = copy.deepcopy(hparams_base)
    hparams['n_tr_qrs'] = n_q
    hparams['run_name'] = f'second_round/{n_q}_qrs'
    hparams['val_check_interval'] = n_q * 10 / 2
    hparams_namespace = Namespace(**hparams)
    run_longformer_marco.main(hparams_namespace)

Training on 80 queries
Namespace(clf_dropout=-1.0, data_dir='drive/MyDrive/info_retrieval/data/', data_loader_bs=1, epochs=6, gpus=1, lr=3e-05, max_seq_len=4096, model_name='allenai/longformer-base-4096', n_tr_qrs=80, num_nodes=1, num_training_steps=120000, num_warmup_steps=1, num_workers=0, res_dir='drive/MyDrive/info_retrieval/results/', run_name='second_round/80_qrs', trainer_batch_size=5, use_10_percent_of_dev=1, use_tensorboard=0, use_wandb=0, val_check_interval=400.0, val_data_loader_bs=1)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

...


DEV:: avg-LOSS: 0.347181499004364 || MRR: 0.5985714285714285 || MRR@10: 0.5985714285714285


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 640: 'val_loss' was not in top 1



DEV:: avg-LOSS: 0.3476015329360962 || MRR: 0.49511904761904757 || MRR@10: 0.49511904761904757


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


#TESTING


In [ ]:
import torch
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from argparse import Namespace

## Testing on Zero shot setting

In [ ]:
hparams['n_tr_qrs'] = 0
hparams['run_name'] = '0_qrs'
hparams_namespace = Namespace(**hparams)
device = 'cuda'
model = TransformerMarco.TransformerMarco(hparams_namespace)
model.to(device)
model.eval()
trainer = Trainer(gpus=hparams_namespace.gpus)
dataloader = model.test_dataloader()
trainer.test(model, dataloaders=dataloader)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight'

OK


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


19474


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.16975
0.16975


[{}]

## Testing on different settings

### Multiple queries (60, 80, ...)

In [ ]:
hparams = {
    'max_seq_len':4096,
    'lr':3e-5,
    'num_warmup_steps':1,
    'num_training_steps':120000,
    'val_check_interval':300,
    'clf_dropout':-1.0,
    'num_workers':0,
    'data_dir':'drive/MyDrive/info_retrieval/data/',
    'trainer_batch_size':5,
    'data_loader_bs':1,
    'val_data_loader_bs':1,
    'use_10_percent_of_dev':1,
    'use_wandb':0,
    'use_tensorboard':0,
    'gpus':1,
    'num_nodes':1,
    'model_name':'allenai/longformer-base-4096',
    'epochs':15,
    'run_name':'80_qrs_all',
    'n_tr_qrs':80,
    'res_dir':'drive/MyDrive/info_retrieval/results/'
}
hparams_namespace = Namespace(**hparams)
#run_longformer_marco.main(hparams)

This works. Change to test on 100queries.

In [ ]:
model = TransformerMarco.TransformerMarco(hparams_namespace)
trainer = pl.Trainer()
chk_path = "/content/drive/MyDrive/info_retrieval/checkpoints/epoch=3-step=560_80qrs_2.ckpt"
checkpoint = torch.load(chk_path)
model.load_state_dict(checkpoint["state_dict"])

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 

<All keys matched successfully>

In [ ]:
device = 'cuda'
model.to(device)
model.eval()
trainer = Trainer(gpus=hparams_namespace.gpus)
dataloader = model.test_dataloader()
trainer.test(model, dataloaders=dataloader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


OK


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


19474


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.20229166666666665
0.20229166666666665


[{}]